<a href="https://colab.research.google.com/github/21Akame03/STELLA/blob/Learning-pytorch/pytorch_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib as plt

In [ ]:
# get training and testing data
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
# create Dataloaders
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for x, y in test_dataloader:
  print(f"Shape of X: {x.shape}")
  print(f"Shape of Y: {y.shape} {y.dtype}")
  break

Shape of X: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64


In [ ]:
# get device
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# defining model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_Relu_stack = nn.Sequential(
      nn.Linear(28*28, 512),
      nn.ReLU(),
      nn.Linear(512, 512),
      nn.ReLU(),
      nn.Linear(512, 10),
      nn.ReLU()
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_Relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_Relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [ ]:
# model parameters
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train(dataloader, model, lossfn, optimiser):
  size = len(dataloader.dataset)
  for batch, (x, y) in enumerate(dataloader):
    x, y = x.to(device), y.to(device)

    # compute prediction error (lossfn)
    pred = model(x)
    loss = lossfn(pred, y)

    # backpropagate
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(x)
      print(f"Loss : {loss :> 7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model):
  size = len(dataloader.dataset)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")

In [ ]:
epochs = 15
for t in range(epochs):
  print(f"Epoch {t+1}\n.....................................")
  train(train_dataloader, model, loss_fn, optimiser)
  test(test_dataloader, model)

print("done")

Epoch 1
.....................................
Loss :  1.043136 [    0/60000]
Loss :  1.143328 [ 6400/60000]
Loss :  0.894469 [12800/60000]
Loss :  1.004615 [19200/60000]
Loss :  1.048600 [25600/60000]
Loss :  1.132886 [32000/60000]
Loss :  0.944417 [38400/60000]
Loss :  1.079136 [44800/60000]
Loss :  1.086073 [51200/60000]
Loss :  1.077822 [57600/60000]
Test Error: 
 Accuracy: 67.4%, Avg loss: 0.016477
Epoch 2
.....................................
Loss :  1.035286 [    0/60000]
Loss :  1.132217 [ 6400/60000]
Loss :  0.885199 [12800/60000]
Loss :  0.998634 [19200/60000]
Loss :  1.042872 [25600/60000]
Loss :  1.125162 [32000/60000]
Loss :  0.935933 [38400/60000]
Loss :  1.074661 [44800/60000]
Loss :  1.076378 [51200/60000]
Loss :  1.068558 [57600/60000]
Test Error: 
 Accuracy: 67.5%, Avg loss: 0.016344
Epoch 3
.....................................
Loss :  1.026448 [    0/60000]
Loss :  1.122433 [ 6400/60000]
Loss :  0.876949 [12800/60000]
Loss :  0.993317 [19200/60000]
Loss :  1.037435 [

In [ ]:
torch.save(model.state_dict(), "data/model.pth")
print("saved")

model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

saved


<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

for i in range(len(test_data)):
  x, y = test_data[i][0], test_data[i][1]
  with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}  |  Actual: {actual}", end="\n\n")

Streaming output truncated to the last 5000 lines.
Predicted: Sneaker  |  Actual: Sandal

Predicted: Dress  |  Actual: Dress

Predicted: Ankle boot  |  Actual: Ankle boot

Predicted: Dress  |  Actual: Dress

Predicted: Sandal  |  Actual: Sandal

Predicted: Shirt  |  Actual: Coat

Predicted: Shirt  |  Actual: T-shirt/top

Predicted: Shirt  |  Actual: Shirt

Predicted: Ankle boot  |  Actual: Ankle boot

Predicted: Ankle boot  |  Actual: Sandal

Predicted: Pullover  |  Actual: Coat

Predicted: Pullover  |  Actual: Pullover

Predicted: Dress  |  Actual: Dress

Predicted: Bag  |  Actual: Bag

Predicted: Sandal  |  Actual: Sandal

Predicted: Bag  |  Actual: Bag

Predicted: Dress  |  Actual: Dress

Predicted: Shirt  |  Actual: Shirt

Predicted: Pullover  |  Actual: Coat

Predicted: Trouser  |  Actual: Trouser

Predicted: Shirt  |  Actual: Coat

Predicted: Ankle boot  |  Actual: Ankle boot

Predicted: Sandal  |  Actual: Sandal

Predicted: Bag  |  Actual: Bag

Predicted: Ankle boot  |  Actual: 